# Imports and utility functions

In [ ]:
!pip install datasets
!pip install transformers

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import transformers as tr
from transformers import DistilBertTokenizerFast
from datasets import load_metric
import torch
import re

     |████████████████████████████████| 325 kB 6.6 MB/s 
     |████████████████████████████████| 136 kB 13.2 MB/s 
     |████████████████████████████████| 77 kB 4.4 MB/s 
     |████████████████████████████████| 212 kB 25.6 MB/s 
     |████████████████████████████████| 1.1 MB 15.8 MB/s 
     |████████████████████████████████| 127 kB 36.5 MB/s 
     |████████████████████████████████| 271 kB 17.1 MB/s 
     |████████████████████████████████| 144 kB 34.2 MB/s 
     |████████████████████████████████| 94 kB 3.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 4.0 MB 9.9 MB/s 


In [ ]:
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
def cleanTexts(texts):
    cleaned = []
    pattern = "[^a-zA-Z0-9]"
    for text in texts:
        clrd = re.sub(pattern," ",text).lower().strip()
        cleaned.append(clrd)
    return cleaned

# Load data

In [ ]:
from sklearn.utils import shuffle

dataset = pd.read_csv('train_data_tweetsENG.csv')
dataset = shuffle(dataset)
dataset.tail()

,Unnamed: 0,text,label
23556,22197,Rock star John Norman Howard (Kris Kristoffers...,0
15394,3228,The story of this film is truly remarkable. A ...,1
14958,17436,I remember back when I was little when I was a...,0
12360,101,The Booth puts a whole new twist on your typic...,1
6867,17744,"I went into this film with expectations, from ...",0


In [ ]:
x_train = list(cleanTexts(dataset['text']))
# print(x[:5])

y_train = list(dataset['label'])
# print(y[:5])

#x_train = x_train[:20000]
#y_train = y_train[:20000]

x_train = x_train[:40000]
y_train = y_train[:40000]

In [ ]:
test_data = pd.read_csv('test_data_tweetsENG.csv')

x_test = list(cleanTexts(test_data['text']))
y_test = list(test_data['label'])

x_test = x_test[:10000]
y_test = y_test[:10000]

# Tokenize data

- **do not forget** to uncomment tokenizer for transformer you are using!

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.20, random_state=69)

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(x_train, truncation=True, padding=True)
val_encodings = tokenizer(x_val,truncation=True, padding=True)
test_encodings = tokenizer(x_test, truncation=True, padding=True)

In [ ]:
class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CreateDataset(train_encodings, y_train)
val_dataset = CreateDataset(val_encodings, y_val)
test_dataset = CreateDataset(test_encodings, y_test)

# BERT

## 1.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

## 2.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoModelForSequenceClassification



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

# DistilBERT

## 3.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

## 4.Experiment

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments



training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="steps",
    eval_steps = 500,
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=32,  # batch size per device during training
    per_device_eval_batch_size=32,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    load_best_model_at_end=True,
)

model = BertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,             # evaluation dataset
    compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
trainer.predict(test_dataset)

# Save model

In [ ]:
torch.save(model, 'modelSeventh')